In [186]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import plotly.express as px
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA

In [65]:
sales_2022 = pd.read_csv("./Data/Iowa_Liquor_Sales_2022.csv")

In [296]:
sales_2022_filtered = sales_2022[['Vendor Name', 'Category Name', 'Bottle Volume (ml)', 'Bottles Sold', 'Sale (Dollars)']]
sales_2022_agg = sales_2022_filtered.groupby(['Vendor Name', 'Category Name', 'Bottle Volume (ml)']).agg(
    cumulative_vol = pd.NamedAgg(column='Bottles Sold', aggfunc='sum')
).reset_index(drop=False)

In [297]:
sales_2022_agg['Agg_Name'] = sales_2022_agg['Category Name'] + "_" + sales_2022_agg['Bottle Volume (ml)'].astype(str)
table = pd.pivot_table(sales_2022_agg, values='cumulative_vol', index=['Vendor Name'],
                       columns=['Agg_Name'], aggfunc="sum", fill_value=0)
table = table.reset_index(drop=False).set_index(['Vendor Name'])

imp_categories = list(sales_2022_agg[sales_2022_agg['Vendor Name'] == 'BACARDI USA INC']['Agg_Name'].unique())
table = table[imp_categories].reset_index(drop=False)
table

Agg_Name,Vendor Name,100% AGAVE TEQUILA_50,100% AGAVE TEQUILA_100,100% AGAVE TEQUILA_200,100% AGAVE TEQUILA_375,100% AGAVE TEQUILA_750,100% AGAVE TEQUILA_1750,AGED DARK RUM_750,AGED DARK RUM_1750,AMERICAN DISTILLED SPIRITS SPECIALTY_750,...,TEMPORARY & SPECIALTY PACKAGES_800,TEMPORARY & SPECIALTY PACKAGES_850,TEMPORARY & SPECIALTY PACKAGES_1750,TENNESSEE WHISKIES_750,WHITE RUM_50,WHITE RUM_200,WHITE RUM_375,WHITE RUM_750,WHITE RUM_1000,WHITE RUM_1750
0,173 CRAFT DISTILLERY,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,228,0,0
1,21ST CENTURY SPIRITS LLC,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3 BADGE CORPORATION,0,0,0,0,50,0,116,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"3-OAKS DISTILLERY, LLC",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,45TH PARALLEL SPIRITS,0,0,0,0,0,0,0,0,6,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
232,WMC-PBD LLC/ PINE BLUFFS DISTILLING,0,0,0,0,0,0,0,0,18,...,0,0,0,0,0,0,0,0,0,0
233,WOODY CREEK DISTILLERS LLC,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
234,WORLD TRADE LIQUOR IMPORTS LLC,0,0,0,0,727,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
235,"WORLDWIDE LIBATIONS, LLC",0,0,0,0,0,0,3,0,0,...,0,0,0,0,0,0,0,18,0,0


In [298]:
table.describe().T.sort_values('count', ascending=False)

,count,mean,std,min,25%,50%,75%,max
Agg_Name,,,,,,,,
100% AGAVE TEQUILA_50,237.0,55.160338,514.409180,0.0,0.0,0.0,0.0,7241.0
100% AGAVE TEQUILA_100,237.0,62.616034,800.886363,0.0,0.0,0.0,0.0,12013.0
IMPORTED VODKAS_375,237.0,253.759494,2552.121307,0.0,0.0,0.0,0.0,32322.0
IMPORTED VODKAS_750,237.0,1123.928270,7781.123742,0.0,0.0,0.0,0.0,101250.0
IMPORTED VODKAS_1000,237.0,430.569620,3768.876647,0.0,0.0,0.0,0.0,42557.0
...,...,...,...,...,...,...,...,...
IMPORTED DISTILLED SPIRITS SPECIALTY_750,237.0,67.535865,393.215002,0.0,0.0,0.0,0.0,3743.0
IMPORTED DRY GINS_50,237.0,0.092827,1.108438,0.0,0.0,0.0,0.0,16.0
IMPORTED DRY GINS_750,237.0,451.742616,3238.660536,0.0,0.0,0.0,0.0,38711.0


In [349]:
scaler = MinMaxScaler()
X = scaler.fit_transform(np.array(table.drop(['Vendor Name'], axis=1)))

tsne = TSNE(n_components=2, perplexity=15)
embed_fit = tsne.fit_transform(X)

names = list(table['Vendor Name'])

history = []
for i in range(20):
    kmeans = KMeans(n_clusters=i+1, random_state=0, n_init='auto').fit(embed_fit)
    labels = kmeans.labels_
    inertia = kmeans.inertia_
    history.append([(i+1), labels, inertia])

history = pd.DataFrame(history, columns=['n_clusters', 'labels', 'inertia'])
history

,n_clusters,labels,inertia
0,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",70496.250000
1,2,"[0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, ...",41484.804688
2,3,"[0, 2, 0, 2, 2, 1, 2, 0, 0, 2, 2, 1, 1, 2, 0, ...",28134.673828
3,4,"[3, 2, 3, 2, 2, 0, 2, 3, 3, 2, 2, 1, 1, 2, 0, ...",22806.513672
4,5,"[3, 2, 3, 2, 2, 4, 2, 3, 3, 2, 0, 1, 1, 2, 4, ...",17397.933594
5,6,"[3, 2, 3, 2, 2, 4, 2, 3, 3, 2, 0, 1, 1, 2, 5, ...",13726.029297
6,7,"[0, 2, 3, 6, 6, 4, 6, 3, 3, 2, 0, 1, 1, 2, 5, ...",10659.005859
7,8,"[0, 2, 0, 4, 4, 3, 4, 0, 5, 2, 6, 1, 7, 2, 5, ...",10679.572266
8,9,"[0, 2, 8, 4, 4, 3, 4, 8, 8, 2, 0, 1, 6, 2, 5, ...",8170.953613
9,10,"[0, 2, 8, 4, 4, 3, 4, 8, 8, 2, 6, 1, 9, 2, 5, ...",7377.158203


In [350]:
px.line(history, x='n_clusters', y='inertia')
# Looks like 6 clusters forms an elbow.

In [363]:
labels = history.loc[history['n_clusters'] == 4]['labels'].values[0]

data_labels = []
try:
    for i in range(len(names)):
        data_labels.append([names[i], labels[i], embed_fit[i][0], embed_fit[i][1], embed_fit[i][2]])

    data_labels = pd.DataFrame(data_labels, columns=['Brand', 'label', "X", "Y", "Z"])
except IndexError:
    for i in range(len(names)):
        data_labels.append([names[i], labels[i], embed_fit[i][0], embed_fit[i][1]])

    data_labels = pd.DataFrame(data_labels, columns=['Brand', 'label', "X", "Y"])

data_labels

,Brand,label,X,Y
0,173 CRAFT DISTILLERY,3,5.905040,13.891884
1,21ST CENTURY SPIRITS LLC,2,14.277925,4.328492
2,3 BADGE CORPORATION,3,9.633894,20.597061
3,"3-OAKS DISTILLERY, LLC",2,13.246187,-6.646710
4,45TH PARALLEL SPIRITS,2,14.011823,-8.463009
...,...,...,...,...
232,WMC-PBD LLC/ PINE BLUFFS DISTILLING,2,3.027181,-6.797962
233,WOODY CREEK DISTILLERS LLC,2,13.757389,-7.057111
234,WORLD TRADE LIQUOR IMPORTS LLC,0,-14.723955,15.020766
235,"WORLDWIDE LIBATIONS, LLC",0,-6.724672,5.764702


In [364]:
n = int(data_labels.loc[data_labels['Brand'] == "BACARDI USA INC"]['label'].values)
display(data_labels.loc[data_labels['Brand'] == "BACARDI USA INC"])
display(data_labels.loc[data_labels['label'] == n])

,Brand,label,X,Y
11,BACARDI USA INC,1,-16.708693,-19.396179


,Brand,label,X,Y
11,BACARDI USA INC,1,-16.708693,-19.396179
12,BAD BEAR ENTERPRISES LLC / LEGENDARY RYE,1,0.117434,-12.635992
20,"BLUE OX SPIRITS, INC.",1,-12.952186,-20.695669
22,"BONAVITA BEVERAGE GROUP, LLC",1,-3.128511,-22.673777
26,BRECKENRIDGE DISTILLERY / DOUBLE DIAMOND DISTI...,1,-3.934476,-23.561373
...,...,...,...,...
222,VINO.COM DBA TOTAL BEVERAGE SOLUTION,1,-17.652910,-4.718464
223,W J DEUTSCH & SONS LTD,1,-3.179633,-24.739395
224,WESTERN SPIRITS BEVERAGE,1,-6.587813,-15.252764
229,WILLIAM GRANT & SONS INC,1,-13.675378,-12.477396


In [365]:
try:
    fig = px.scatter_3d(data_labels, x="X", y="Y", z='Z', color='label', hover_data=['Brand'])
except ValueError:
    fig = px.scatter(data_labels, x="X", y="Y", color='label', hover_data=['Brand'])

fig.show()